![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/reader/SparkNLP_PDFToText_Annotator_Demo.ipynb)

# Introducing PDFToText annotator in SparkNLP
This notebook showcases the newly added  `PDFToText` method in Spark NLP that parses PDF content from both local files and distributed file systems into a Spark DataFrame.

## Setup and Initialization
Let's keep in mind a few things before we start 😊

Support for reading pdf files was introduced in Spark NLP 6.0.0 Please make sure you have upgraded to the latest Spark NLP release.

Let's install and setup Spark NLP in Google Colab. This part is pretty easy via our simple script

In [1]:
! wget -q http://setup.johnsnowlabs.com/colab.sh -O - | bash

Mounted at /content/drive
Processing ./spark_nlp-6.0.0-py2.py3-none-any.whl
Apache Spark version: 3.5.5


For local files example we will download a couple of PDF files from Spark NLP Github repo:

In [2]:
!mkdir pdf-files
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/pdf/pdf-title.pdf -P pdf-files
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/pdf/text_3_pages.pdf -P pdf-files

--2025-04-29 08:44:49--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/pdf/pdf-title.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25803 (25K) [application/octet-stream]
Saving to: ‘pdf-files/pdf-title.pdf’

pdf-title.pdf       100%[===================>]  25.20K  --.-KB/s    in 0s      

2025-04-29 08:44:49 (114 MB/s) - ‘pdf-files/pdf-title.pdf’ saved [25803/25803]

--2025-04-29 08:44:49--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/src/test/resources/reader/pdf/text_3_pages.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443...

## Parsing PDFs from Local Files
Use the `PdfToText()` annotator to parse Excel content from local directories.

In [3]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

We need to set the configuraiton below. This setting is primarily included for backward compatibility with older versions of Spark.

In [4]:
spark.conf.set("spark.sql.legacy.allowUntypedScalaUDF", "true")

In [6]:
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.reader.pdf_to_text import *

pdf_to_text = PdfToText().setStoreSplittedPdf(True)
test_df = spark.read.format("binaryFile").load("./pdf-files")

In [7]:
pipeline = Pipeline(stages=[pdf_to_text])
pipeline_model = pipeline.fit(test_df)
pdf_df = pipeline_model.transform(test_df)
pdf_df.show()

+--------------------+--------------------+------+------------------------------+----------------+---------------+--------------------+---------+-------+
|                path|    modificationTime|length|PdfToText_d3ee37ae1218__output|height_dimension|width_dimension|             content|exception|pagenum|
+--------------------+--------------------+------+------------------------------+----------------+---------------+--------------------+---------+-------+
|file:/content/pdf...|2025-04-29 08:44:...| 25803|          This is a Title \...|             842|            596|[25 50 44 46 2D 3...|     NULL|      0|
|file:/content/pdf...|2025-04-29 08:44:...|  9487|          This is a page.\n...|             841|            595|[25 50 44 46 2D 3...|     NULL|      0|
+--------------------+--------------------+------+------------------------------+----------------+---------------+--------------------+---------+-------+



In [8]:
pdf_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- PdfToText_d3ee37ae1218__output: string (nullable = true)
 |-- height_dimension: integer (nullable = true)
 |-- width_dimension: integer (nullable = true)
 |-- content: binary (nullable = true)
 |-- exception: string (nullable = true)
 |-- pagenum: integer (nullable = true)



You can also use DFS file systems like:
- Databricks: `dbfs://`
- HDFS: `hdfs://`
- Microsoft Fabric OneLake: `abfss://`